## Deep Learning Completely Inelastic Collision Solver

This notebook implements a recurrent neural network to solve for the post collision velocity of a two-mass system where the collision is completely inelastic and the second mass is initially at rest. The model is formulated using a recurrent neural network (RNN) and its input is represented as a text string giving the masses and the initial velocity of the first mass. Exact solutions calculated using conservation of momentum equations are used to train the RNN and evaluate the accuracy of predictions. This notebook is inspired by the addition_rnn.py example included with Keras.

### Import useful packaged including TensorFlow 2.0 and Keras.
The notebook utilizes tensorflow >= 2.0, which now includes keras, a package of high level wrappers designed to make building and training deep learning models easier. 

In [1]:
# import packages
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from tensorflow.keras.layers import RNN, LSTM, TimeDistributed, RepeatVector, Dense, LSTMCell, Dropout
# LSTM doesn't work well since cuDNN is compiled with certain restrictions. 
# So, here I will create LSTM layers by wrapping LSTMCell in RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

### Check to see if a Tensorflow is installed with GPU support and if a GPU is available.

In [2]:
if not tf.test.is_gpu_available():
    print('No GPU found. Training will be slower.')
else:
    print('Default GPU {} found.'.format(tf.test.gpu_device_name()))

Default GPU /device:GPU:0 found.


### Define a class to encode and decode between a selection of characters and one-hot integer representations.

In [3]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)
    
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

### Generate input and output character sequences.

The input is in the form {}m{}v{} where the values in the {}s are randomly drawn one or two character sequences representing, from left to right, the mass of object one, the mass of object two, and the initial velocity of object one. Object two is initially at rest.

The output is a character string representing the post collision velocity of the system after a head on totally inelastic collision.

In [4]:
# Parameters for the model and dataset.
num_problems = 80000 # number of sequences in the training and validation sets
digits = 2 # maximum number of digits for mass and velocity in the input sequence 
lenans = 4 # number of characters in the answer sequence 

REVERSE=False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
maxlen = digits + 1 + digits + 1 + digits

# All the numbers, plus sign and space for padding.
chars = '0123456789mv. '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < num_problems:
    f = lambda: int(''.join(np.random.choice(list('123456789'))
                    for i in np.arange(np.random.randint(1, digits + 1))))
    a, b, c = f(), f(), f()
    # Skip any questions we've already seen
    key = tuple(sorted((a, b, c)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}m{}v{}'.format(a, b, c)
    query = q + ' ' * (maxlen - len(q))
    ans = a*c/(a+b)
    if ans < 10:
        r = lenans-2
    elif ans < 100:
        r = lenans-3
    ans = str(round(a*c/(a+b),r))
    # Answers can be of maximum size LENANS.
    ans += '0' * (lenans - len(ans))
    questions.append(query)
    expected.append(ans)
print('Total momentum questions:', len(questions))

print('Vectorization...')
x = np.zeros((len(questions), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), lenans, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, maxlen)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, lenans)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# set apart 20% for validation and text data that we never train over.
split_at = len(x) - len(x) // 5
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

split_at =  len(x_val) - len(x_val) // 2 
(x_val, x_test) = x_val[:split_at], x_val[split_at:]
(y_val, y_test) = y_val[:split_at], y_val[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Test Data:')
print(x_test.shape)
print(y_test.shape)

Generating data...
Total momentum questions: 80000
Vectorization...
Training Data:
(64000, 8, 14)
(64000, 4, 14)
Validation Data:
(8000, 8, 14)
(8000, 4, 14)
Test Data:
(8000, 8, 14)
(8000, 4, 14)


In [5]:
# show an example of the input and output character strings
ii = np.random.randint(0, 1000)
print('input:', questions[ii], 'output:', expected[ii])

input: 1m9v93   output: 9.30


In [6]:
# hyperparameters 
HIDDEN_SIZE = 128
BATCH_SIZE = 256
LAYERS = 2
learning_rate = 0.001

print('Build model...')
model = Sequential()
# "encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# note: in a situation where your input sequences have a variable length, use input_shape=(None, num_feature).
# model.add(LSTM(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(RNN(LSTMCell(HIDDEN_SIZE), input_shape=(maxlen, len(chars))))

# as the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'lenans' times as that's the maximum length of output.
model.add(RepeatVector(lenans))
# the decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # by setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    # model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
    model.add(RNN(LSTMCell(HIDDEN_SIZE), return_sequences=True))

# add a dropout layer to prevent overfitting
# model.add(Dropout(rate=0.1))
# add a dense layer to every temporal slice of an input. for each of step of the output sequence, 
# decide which character should be chosen.
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              # optimizer=keras.optimizers.Adam(lr=learning_rate),
              optomizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
model.summary()


Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, 128)               73216     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
rnn_1 (RNN)                  (None, 4, 128)            131584    
_________________________________________________________________
rnn_2 (RNN)                  (None, 4, 128)            131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 4, 14)             1806      
Total params: 338,190
Trainable params: 338,190
Non-trainable params: 0
_________________________________________________________________


### Train the model and generate example predictions every 3 epochs.

In [7]:
# train the model and show predictions against the validation dataset.
for iteration in range(1, 36):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=3,
              validation_data=(x_val, y_val))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('score:', score)
    # select 10 samples from the validation set at random so we can visualize errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_test[np.array([ind])], y_test[np.array([ind])]
        preds = model.predict_classes(1.0*rowx, verbose=0) # mult by 1.0 to get data types to jive
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('A', correct, end=' ')
        
        for ii in np.arange(len(correct)):
            if correct[ii] == guess[ii]:
                print(colors.ok + '☑' + colors.close, end='')
            else:
                print(colors.fail + '☒' + colors.close, end='')
        
        print(' P', guess)


--------------------------------------------------
Iteration 1
Train on 64000 samples, validate on 8000 samples
Epoch 1/3
64000/64000 [==============================] - 6s 91us/sample - loss: 2.0252 - accuracy: 0.3221 - val_loss: 1.9072 - val_accuracy: 0.3399
Epoch 2/3
64000/64000 [==============================] - 2s 39us/sample - loss: 1.8240 - accuracy: 0.3581 - val_loss: 1.6968 - val_accuracy: 0.3859
Epoch 3/3
64000/64000 [==============================] - 2s 39us/sample - loss: 1.6521 - accuracy: 0.3887 - val_loss: 1.5840 - val_accuracy: 0.3912
score: [1.5855826959609984, 0.39]
Q 3m92v97  A 3.06 ☒☑☒☒ P 1.30
Q 94m53v88 A 56.3 ☒☒☑☒ P 44.6
Q 74m74v4  A 2.00 ☒☑☒☒ P 1.38
Q 29m32v31 A 14.7 ☑☒☑☒ P 11.3
Q 82m7v32  A 29.5 ☑☒☑☒ P 24.8
Q 59m98v4  A 1.50 ☒☑☒☒ P 0.33
Q 4m89v74  A 3.18 ☒☑☒☑ P 1.38
Q 77m16v75 A 62.1 ☒☒☑☒ P 54.6
Q 81m77v49 A 25.1 ☒☒☑☒ P 14.3
Q 41m9v74  A 60.7 ☒☒☑☒ P 58.8

--------------------------------------------------
Iteration 2
Train on 64000 samples, validate on 8000 samp

Q 14m89v97 A 13.2 ☑☑☑☒ P 13.1
Q 22m54v78 A 22.6 ☑☑☑☒ P 22.3
Q 98m88v24 A 12.6 ☑☑☑☒ P 12.1

--------------------------------------------------
Iteration 8
Train on 64000 samples, validate on 8000 samples
Epoch 1/3
64000/64000 [==============================] - 2s 38us/sample - loss: 1.0580 - accuracy: 0.5813 - val_loss: 1.1565 - val_accuracy: 0.5343
Epoch 2/3
64000/64000 [==============================] - 2s 38us/sample - loss: 1.0475 - accuracy: 0.5848 - val_loss: 1.0371 - val_accuracy: 0.5832
Epoch 3/3
64000/64000 [==============================] - 2s 38us/sample - loss: 1.0366 - accuracy: 0.5898 - val_loss: 1.0094 - val_accuracy: 0.5998
score: [1.0074513502120972, 0.6014063]
Q 99m23v56 A 45.4 ☑☒☑☒ P 44.5
Q 61m2v94  A 91.0 ☑☑☑☒ P 91.1
Q 46m97v44 A 14.2 ☑☒☑☒ P 13.9
Q 64m89v62 A 25.9 ☑☑☑☒ P 25.5
Q 91m1v27  A 26.7 ☑☑☑☒ P 26.6
Q 8m18v43  A 13.2 ☑☒☑☒ P 12.0
Q 38m2v71  A 67.5 ☑☒☑☒ P 68.9
Q 22m8v73  A 53.5 ☑☒☑☑ P 54.5
Q 1m51v22  A 0.42 ☑☑☒☒ P 0.25
Q 41m34v63 A 34.4 ☑☒☑☒ P 33.5

-------------

64000/64000 [==============================] - 2s 38us/sample - loss: 0.8488 - accuracy: 0.6674 - val_loss: 0.8952 - val_accuracy: 0.6420
Epoch 2/3
64000/64000 [==============================] - 2s 38us/sample - loss: 0.8433 - accuracy: 0.6698 - val_loss: 0.8314 - val_accuracy: 0.6712
Epoch 3/3
64000/64000 [==============================] - 2s 38us/sample - loss: 0.8416 - accuracy: 0.6701 - val_loss: 0.8651 - val_accuracy: 0.6618
score: [0.866441558599472, 0.6590313]
Q 22m54v57 A 16.5 ☑☑☑☒ P 16.0
Q 8m35v13  A 2.42 ☑☑☒☒ P 2.20
Q 41m48v8  A 3.69 ☑☑☒☒ P 3.50
Q 52m74v12 A 4.95 ☑☑☒☒ P 4.30
Q 78m46v29 A 18.2 ☑☑☑☒ P 18.8
Q 2m89v29  A 0.64 ☑☑☒☒ P 0.53
Q 28m41v73 A 29.6 ☑☑☑☒ P 29.3
Q 75m3v92  A 88.5 ☑☑☑☑ P 88.5
Q 37m81v79 A 24.8 ☑☑☑☒ P 24.3
Q 71m14v5  A 4.18 ☑☑☑☑ P 4.18

--------------------------------------------------
Iteration 22
Train on 64000 samples, validate on 8000 samples
Epoch 1/3
64000/64000 [==============================] - 2s 39us/sample - loss: 0.8392 - accuracy: 0.6709 - val_lo

Q 24m72v59 A 14.8 ☑☒☑☒ P 15.2
Q 67m91v44 A 18.7 ☑☑☑☒ P 18.9
Q 53m59v28 A 13.2 ☑☑☑☒ P 13.6

--------------------------------------------------
Iteration 28
Train on 64000 samples, validate on 8000 samples
Epoch 1/3
64000/64000 [==============================] - 2s 38us/sample - loss: 0.7933 - accuracy: 0.6895 - val_loss: 0.8693 - val_accuracy: 0.6583
Epoch 2/3
64000/64000 [==============================] - 2s 38us/sample - loss: 0.7918 - accuracy: 0.6907 - val_loss: 0.7983 - val_accuracy: 0.6832
Epoch 3/3
64000/64000 [==============================] - 2s 38us/sample - loss: 0.7881 - accuracy: 0.6927 - val_loss: 0.8026 - val_accuracy: 0.6790
score: [0.8030582339763641, 0.6786875]
Q 25m29v65 A 30.1 ☑☑☑☒ P 30.6
Q 66m24v5  A 3.67 ☑☑☑☒ P 3.66
Q 51m96v93 A 32.3 ☑☑☑☒ P 32.0
Q 68m81v19 A 8.67 ☑☑☑☒ P 8.60
Q 7m22v43  A 10.4 ☑☑☑☒ P 10.6
Q 25m53v75 A 24.0 ☑☒☑☒ P 23.3
Q 35m35v8  A 4.00 ☒☑☒☑ P 3.90
Q 58m59v98 A 48.6 ☑☒☑☒ P 49.5
Q 37m47v85 A 37.4 ☑☑☑☒ P 37.0
Q 62m15v3  A 2.42 ☑☑☑☒ P 2.40

------------